In [97]:
import numpy as np
from re import match, compile
 
human_dirs = np.array(range(-1,2))
distances =  np.array(range(-1,2))
velocities = np.array(range(-1,2))
robot_dirs = np.array(range(-1,2))

    
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)


def dim2str(d):
    if d > 0:
        return '+'
    elif d < 0:
        return '-'
    else:
        return '0'

def state2str(s):
    return ''.join([dim2str(d) for d in s])
    

valid_pattern = [
    ('-...', '0...'),
]

valid_pattern = [
    ('00', '..'),
    ('-.', '0.'),
    ('+.', '0.'),
    ('.-', '.0'),
    ('.+', '.0')
]

#valid_pattern_inv = [ (f[1], f[0]) for f in valid_pattern] 

#valid_pattern = valid_pattern + valid_pattern_inv

def fill_trans(cp):
    trans = np.zeros((len(cp), len(cp)), dtype=bool)
    for p in valid_pattern:
        pat = p[0].replace('.','(.)').replace('+', '\+')
        r = compile(pat)
        for i_f in range(0, len(cp)):
            f = cp[i_f]
            fs = state2str(f)
            m = r.match(fs)
            if m:
                to_pat = list(p[1])
                count_i = -1
                for pos in range(0,len(to_pat)):
                    if to_pat[pos] == '.':
                        #try:
                        count_i += 1
                        if count_i < len(m.groups()):
                            to_pat[pos] = m.groups()[count_i]
                        #except:
                        #    pass
                #print(to_pat)
                to_pat = ''.join(to_pat)
                patt = to_pat.replace('+', '\+')
                rt = compile(patt)
                for i_t in range(0, len(cp)):
                    t = cp[i_t]
                    ts = state2str(t)

                    if rt.match(ts):
#                        print('%s => %s' % (
#                            state2str(t),
#                            state2str(f)
#                        ))
                        if not i_f == i_t:
                            print('%s => %s' % (
                                state2str(f),
                                state2str(t)
                            ))
                            trans[i_f, i_t] = True
                            trans[i_t, i_f] = True
    return trans

#cp = cartesian_product(human_dirs, distances, velocities, robot_dirs)
cp = cartesian_product(human_dirs, robot_dirs)

trans = fill_trans(cp)

print(trans.sum().sum())
#for s in cp:
#    print(state2str(s))
#np.where(trans == True)



00 => --
00 => -0
00 => -+
00 => 0-
00 => 0+
00 => +-
00 => +0
00 => ++
-- => 0-
-0 => 00
-+ => 0+
+- => 0-
+0 => 00
++ => 0+
-- => -0
0- => 00
+- => +0
-+ => -0
0+ => 00
++ => +0
32



1. robot dir: a: (-,0,+)
2. human dir: b: (-,0,+)
3. velocity r > h: c: (T, F)
4. in safety distance: d: (T, F)

36 states

# Conditions:

1. always ensure robot not in safety critical distance: [] (a,b,c,d) d != T
[2. robot never faster than human: [] (a,b,c,d) c != T]

# Assumption:

1. human doesn't change 

# Example:
* (-, +, F, F) we can't reach a state that is (., ., ., T)
* (-, +, F, F) -> (0, +, F, F) -> (+, +, F, F) -> (+, +, T, F) -> !!!!(., ., ., T)


## assume 4 states, 1 is not allowed, general transitions:

* 1->2, 2->3, 3->4, 1->4 
* 4 is invalid
* if in 1: 1,2,3,4 or 1,4
* if in 2: 2,3,4

you find the shortest route to invalid states (most critical cause of action)


robot triggered actions:

* 00 - 0- 
* 00 - 0+ 
* -0 - --
* -0 - -+
* +0 - +-
* +0 - ++

human trigger actions

* 00 - -0
* 00 - +0
* 0- - --
* 0- - +-
* 0+ - -+
* 0+ - ++

both (disregard them):
* 00 - --
* 00 - ++
* 00 - +-
* 00 - -+

forbidden: --, 0-

-+ -> 00 (B)
-+ -> 0+ (H)
-+ -> -0 (R)

In [ ]:
%matplotlib inline
import numpy as np
from math import pi, cos, sin
from uuid import uuid4
import matplotlib.pyplot as plt

class Agent():
    def __init__(self, name=None, min_vel=0.1, max_vel=1.0, start_pos=(5, 0), start_pos_std=10.0):
        if name:
            self._name = name
        else:
            self._name = str(uuid4())
        self._dir = np.random.random() * 2.0 * pi
        self._vel = np.random.random() * (max_vel - min_vel) + min_vel
        self._vec = np.array((
            cos(self._dir) * self._vel,
            sin(self._dir) * self._vel,
        ))
        self._pos = np.random.randn(2) + start_pos
        self._traj = [self._pos.copy()]

    def __str__(self):
        return ("%s: dir=%fdeg vel=%fm/s vec=(%f, %f)m/s pos=(%f, %f)m" % (
            self._name,
            self._dir * 180 / pi,
            self._vel,
            self._vec[0], self._vec[1],
            self._pos[0], self._pos[1]
        ))
    
    def step(self, delta_t=1.0):
        self._pos += self._vec * delta_t
        self._traj.append(self._pos.copy())
    
    def traj(self):
        a = np.zeros((len(self._traj), 4))
        a[:,0:2] = np.array(self._traj)
        a[:,2:4] = self._vec
        return a

    
class QSR_pair():
    def __init__(self, 
                 agents=None
                ):
        if agents:
            self._agents= agents
        else:
            self._agents =  [
                     Agent('robot', start_pos=(0,0)),
                     Agent('human', start_pos=(3,0))
                 ]
        self._step = 1
        self._n_steps = 50

    def run(self):
        for t in np.linspace(0,self._n_steps * self._step,self._n_steps+1):
            #print(t)
            #print(robot)
            #print(human)
            for a in self._agents:
                a.step(self._step)
        
    def plot(self, axes):
        for a in self._agents:
            t = a.traj()
            axes.quiver(t[:,0], t[:,1], t[:,2], t[:,3])


pair = QSR_pair()
pair.run()
a=plt.axes()
pair.plot(a)
#print(pair._agents[0].traj())
#a.scatter(robot.traj()[:,0], robot.traj()[:,1])
#a.scatter(human.traj()[:,0], human.traj()[:,1])
    
    